In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# This should be the path to the `train_images.zip` file in your google drive.
!unzip gdrive/My\ Drive/train_images.zip

In [ ]:
for model_name in [
    "model0",
    "model1",
    "model2"
    ]:
  training_dir = f"train_data/{model_name}"
  train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.00)

  train_data = train_datagen.flow_from_directory(training_dir, target_size = (250,250), batch_size = 32, class_mode = 'categorical',
                                          subset = 'training')

  ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'

  model_ResNet = tf.keras.Sequential([
      hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(23, activation = 'softmax', name = 'Output_layer')
  ])

  model_ResNet.compile(
      optimizer = tf.keras.optimizers.Adam(),
      loss = tf.keras.losses.CategoricalCrossentropy(),
      metrics = ['accuracy']
  )


  model_ResNet.summary()
  tf.keras.utils.plot_model(model_ResNet)
  resnet_model = model_ResNet.fit(train_data, epochs = 6, verbose = 1) #5
  model_ResNet.save(f"{model_name}_weights.keras")